## Importing libraries and defining relevant folders

#### Importing libraries

In [1]:
import os, sys, time, datetime, glob, bcolz, random

In [2]:
import shutil

In [3]:
import keras
import numpy as np
import pandas as pd

Using TensorFlow backend.


In [4]:
from keras import Sequential

In [5]:
import matplotlib.pyplot as plt

In [6]:
from keras.preprocessing.image import ImageDataGenerator
from keras import applications
from keras import optimizers
from keras.models import Sequential, Model
from keras.layers import Dropout, Flatten, Dense, Convolution2D, Input, BatchNormalization

In [7]:
from keras_vggface.vggface import VGGFace

#### Relevant folders

In [8]:
folder_affectnet = "/media/amogh/Stuff/CMU/datasets/AffectNet/"

In [9]:
folder_train_data = folder_affectnet + 'Manually_Annotated_Images/'

## Generating labels for valence and arousal training

#### Reading and filtering dataframes from csv files

In [10]:
path_training_csv = folder_affectnet + "Manually_Annotated_file_lists/training_va.csv"
folder_annotated_training_images = folder_affectnet + "Manually_Annotated_Images/"

Getting image_filename and the corresponding labels for valence and arousal

In [11]:
df_train = pd.read_csv(path_training_csv,header=None)
df_train.head()

,0,1,2
0,689/737db2483489148d783ef278f43f486c0a97e140fc...,0.785714,-0.055556
1,392/c4db2f9b7e4b422d14b6e038f0cdc3ecee239b5532...,-0.017253,0.004313
2,468/21772b68dc8c2a11678c8739eca33adb6ccc658600...,0.174603,0.007937
3,944/06e9ae8d3b240eb68fa60534783eacafce2def60a8...,0.153401,0.038890
4,993/02e06ee5521958b4042dd73abb444220609d96f57b...,0.783972,-0.551684


In [12]:
list_image_paths = df_train.iloc[:,0].values

In [13]:
df_va_train = df_train.iloc[:,[0,len(df_train.columns)-2,len(df_train.columns)-1]]
df_va_train.columns = ["image","valence", "arousal"]
print("shape is: ", df_va_train.shape)
(df_va_train.head())

shape is:  (414799, 3)


,image,valence,arousal
0,689/737db2483489148d783ef278f43f486c0a97e140fc...,0.785714,-0.055556
1,392/c4db2f9b7e4b422d14b6e038f0cdc3ecee239b5532...,-0.017253,0.004313
2,468/21772b68dc8c2a11678c8739eca33adb6ccc658600...,0.174603,0.007937
3,944/06e9ae8d3b240eb68fa60534783eacafce2def60a8...,0.153401,0.038890
4,993/02e06ee5521958b4042dd73abb444220609d96f57b...,0.783972,-0.551684


Converting the dataframe in a dictionary

In [14]:
dict_im_va = df_va_train.set_index('image').T.to_dict('list')
dict_im_va

{'689/737db2483489148d783ef278f43f486c0a97e140fc4b6b61b84363ca.jpg': [0.785714,
  -0.055555600000000004],
 '392/c4db2f9b7e4b422d14b6e038f0cdc3ecee239b55326e9181ee4520f9.jpg': [-0.0172533,
  0.00431332],
 '468/21772b68dc8c2a11678c8739eca33adb6ccc658600e4da2224080603.jpg': [0.174603,
  0.00793651],
 '944/06e9ae8d3b240eb68fa60534783eacafce2def60a86042f9b7d59544.jpg': [0.153401,
  0.0388903],
 '993/02e06ee5521958b4042dd73abb444220609d96f57b1689abbe87c024.jpg': [0.783972,
  -0.5516840000000001],
 '979/f675c6a88cdef99a6d8b0261741217a0319387fcf1571a174f99ac81.jpg': [-0.34126999999999996,
  0.547619],
 '637/94b769d8e880cbbea8eaa1350cb8c094a03d27f9fef44e1f4c0fb2ae.jpg': [-2.0,
  -2.0],
 '997/b81f843f08ce3bb0c48b270dc58d2ab8bf5bea3e2262e50bbcadbec2.jpg': [-0.488145,
  0.831839],
 '358/21a32dd1c1ecd57d3e8964621c911df1c0b3348a4ae5203b4a243230.JPG': [-2.0,
  -2.0],
 '330/60216cf5171a1f18876f178e4ab84db22f2dc0920bdaca819769b032.jpg': [-2.0,
  -2.0],
 '981/5c991a28b5550cf5e1e830d5eed1dc7f80581462d627

#### Moving the files with bad labels from the training set

In [87]:
image_name_bad_labels = ([t for t in dict_im_va.keys() if dict_im_va.get(t) == [-2.,-2.]])
len(image_name_bad_labels)

94060

Code to remove images with bad labels: 

In [16]:
len(dict_im_va)

414799

#### Defining function to get labels

In [120]:
def get_va_from_image(image_name, dict_im_va=dict_im_va):
    va = dict_im_va.get(image_name)
    if va==None:
        return(np.zeros(2))
    else:
        return va

## Reading images from folder and saving convolutional features (for now just training)

In [121]:
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input

In [122]:
def get_data(path, target_size=(224,224), class_mode=None, shuffle=False, batch_size=1):
    gen = ImageDataGenerator()
    batches = gen.flow_from_directory(path, target_size=target_size, class_mode=class_mode, shuffle=shuffle, batch_size=batch_size)
    return batches

In [123]:
batches_flow_from_dir = get_data(folder_train_data)

Found 30899 images belonging to 74 classes.


## Defining custom flow_from_directory generator for labels

In [124]:
def flow_from_directory_va(flow_from_directory_gen):
    for x_batch in flow_from_directory_gen:
        idx = (flow_from_directory_gen.batch_index - 1) * flow_from_directory_gen.batch_size
        filenames_batch = flow_from_directory_gen.filenames[idx : idx + flow_from_directory_gen.batch_size]
        labels_batch = np.array([get_va_from_image(f) for f in filenames_batch])
        x_batch = preprocess_input(x_batch)
        yield x_batch, labels_batch

In [125]:
list_train_image_fnames = (glob.glob(folder_train_data+'/*/*'))
list_train_image_fnames

['/media/amogh/Stuff/CMU/datasets/AffectNet/Manually_Annotated_Images/1017/000c10a8f6735212d71eb54298bc47ea56dc08e64ed2cab8913d683f.jpg',
 '/media/amogh/Stuff/CMU/datasets/AffectNet/Manually_Annotated_Images/1017/005f28c1d064467c3a85ce85dcad0b749bd8c377d7cca87f44922ba9.jpg',
 '/media/amogh/Stuff/CMU/datasets/AffectNet/Manually_Annotated_Images/1017/0065d8d262b8ddbb58ec1dedefe0fc44f2b2dc191b8e9a54cb712f78.jpg',
 '/media/amogh/Stuff/CMU/datasets/AffectNet/Manually_Annotated_Images/1017/00c2038b4ddd4ecbc6f253b4c4897a434aafbb146199d84438e02127.png',
 '/media/amogh/Stuff/CMU/datasets/AffectNet/Manually_Annotated_Images/1017/0119706ba67e9d827fdc4eadad32a49c545c936548e2cd2b729d758f.png',
 '/media/amogh/Stuff/CMU/datasets/AffectNet/Manually_Annotated_Images/1017/0186eddce00151f7f9d477cece6add341576e6fcdf5bc7ef3ac1417d.jpg',
 '/media/amogh/Stuff/CMU/datasets/AffectNet/Manually_Annotated_Images/1017/02a0edc13ef09a20afe411a0df89d850cf5ee18dc5e23178561358f7.jpg',
 '/media/amogh/Stuff/CMU/datasets/

# Models - Defining, loading data, training, saving

### Model 1: VGG16 -> model_vgg_reg1

#### Defining the model

In [128]:
vgg_full = applications.VGG16(weights='imagenet', include_top=False, input_shape=(224,224,3))

In [129]:
vgg_full.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

Building the model on top of the conv part

In [130]:
p = 0.6

In [131]:
inp_top_model = Input(vgg_full.layers[-1].output_shape[1:])
x = BatchNormalization(axis=1)(inp_top_model)
x = Dropout(p/4)(x)
x = Flatten()(x)
x = Dense(512, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(p)(x)
x = Dense(512, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(p/2)(x)
x_va = Dense(2, name='bva')(x)

In [132]:
top_model = Model(inp_top_model, x_va)

In [133]:
final_output = top_model(vgg_full.output)

In [134]:
model_vgg_reg1 = Model(vgg_full.input, final_output)

In [135]:
model_vgg_reg1.compile(loss='mean_squared_error',optimizer=optimizers.SGD(lr=0.001), metrics=['accuracy'])

In [136]:
model_vgg_reg1.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

#### Loading batches, making destination folders

In [141]:
gen_va = flow_from_directory_va(batches_flow_from_dir)

#### Training of the model

In [142]:
model_vgg_reg1.fit_generator(gen_va, epochs=3, steps_per_epoch=10000)

Epoch 1/3
   40/10000 [..............................] - ETA: 2:44:26 - loss: 0.1360 - acc: 0.7750

KeyboardInterrupt: 

In [137]:
new_model.summary(ImageDataGenerator())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 1000)              134260544 
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 2002      
Total params: 134,262,546
Trainable params: 134,262,546
Non-trainable params: 0
_________________________________________________________________


#### Unnecessary for now: saving conv features for VGG16

Saving conv features

In [40]:
layers = model_vgg16.layers
layer_idx = [index for index,layer in enumerate(layers) if type(layer) is Convolution2D][-1]
conv_layers, fc_layers = layers[:layer_idx+1], layers[layer_idx+1:]

In [41]:
model_vgg16_conv = Sequential(conv_layers)

In [42]:
model_vgg16_conv.layers

## Model 2: 

In [72]:
vgg_face_full = VGGFace(include_top=False, input_shape=(224, 224, 3), pooling='avg')

58916864/58909280 [==============================] - 22s 0us/step


In [73]:
vgg_face_full.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
conv1_1 (Conv2D)             (None, 224, 224, 64)      1792      
_________________________________________________________________
conv1_2 (Conv2D)             (None, 224, 224, 64)      36928     
_________________________________________________________________
pool1 (MaxPooling2D)         (None, 112, 112, 64)      0         
_________________________________________________________________
conv2_1 (Conv2D)             (None, 112, 112, 128)     73856     
_________________________________________________________________
conv2_2 (Conv2D)             (None, 112, 112, 128)     147584    
_________________________________________________________________
pool2 (MaxPooling2D)         (None, 56, 56, 128)       0         
__________